In [54]:
import numpy as np
import pandas as pd
import re
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_distances
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
human1 = []
human2 = []
lines = []

In [56]:
len(file)

115782

In [57]:
dialogs = []
with open('human_chat.txt', 'r') as w:
    for line in w:
        try:
            dialogs.append(line.split(': ')[1].split("\n")[0])
        except:
            print('done')
        
len(dialogs)

done


1494

In [59]:
for ind, line in enumerate(dialogs):
    if ind%2==0:
        human1.append(line)
    else:
        human2.append(line)

# save to a dataframe
df = pd.DataFrame({'Question':human1, 'Answer':human2})

In [60]:
len(human1), len(human2)

(747, 747)

In [62]:
df.isnull().sum()

Question    0
Answer      0
dtype: int64

In [63]:
stop_words_list = stopwords.words("english")

In [64]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [word for word in words if not word in stop_words_list]
    words = [word.strip() for word in words if len(word.strip()) > 1]
    text = " ".join(words)
    return text

In [65]:
df["Question"] = df["Question"].apply(preprocess)

In [66]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(text):
    stemmed_text = ""
    for word in text.split():
        stem = lemmatizer.lemmatize(word)
        stemmed_text += stem
        stemmed_text += " "
        
    stemmed_text = stemmed_text.strip()
    return stemmed_text

In [67]:
df["Question"] = df["Question"].apply(preprocess)

In [68]:
stemmer = PorterStemmer()
def stemmizing(text):
    stemmed_text = ""
    for word in text.split():
        stem = stemmer.stem(word)
        stemmed_text += stem
        stemmed_text += " "
        
    stemmed_text = stemmed_text.strip()
    return stemmed_text

In [69]:
df["Question"] = df["Question"].apply(preprocess)

In [71]:
analyzer = SentimentIntensityAnalyzer()
df["rating"] = df["Question"].apply(analyzer.polarity_scores)
df = pd.concat([df.drop(["rating"], axis=1), df["rating"].apply(pd.Series)], axis=1)

In [73]:
tfidf = TfidfVectorizer()

In [74]:
factors = tfidf.fit_transform(df["Question"]).toarray()

In [75]:
while True:
    query = input("User#> ")
    if query == "exit":
        break
        
    query = lemmatizing(query)
    query_vector = tfidf.transform([query]).toarray()
    similar_score = 1 - cosine_distances(factors, query_vector)
    index = similar_score.argmax()
    match = df.loc[index]["Question"]
    response = df.loc[index]["Answer"]
    if similar_score[0] <= 0.1:
        print("Bot#> Please rephrase your question.")
        
    else:
        print("Bot#> ", response)

User#> hi
Bot#>  What is your favorite holiday?
User#> exit
